In [16]:
import matplotlib.pyplot as plt
import numpy as np
import random

In [17]:
def csvRead(filepath):
    data = np.genfromtxt(filepath, delimiter=',', skip_header=1)
    return data

darthPlagueis = csvRead('FLIGHT DATA/pcb3_darthplagueis_flightdata.csv')
apple = csvRead('FLIGHT DATA/pcb4_apple_flightdata.csv')

In [18]:
def filterData(array):
    deployedRow = np.where(array[:,0] == 1)[0][0]
    landedRow = np.where(array[:,0] == 4)[0][0]
    return array[deployedRow-250:landedRow+100]

def filterData2(array, deployedRow, landedRow):
    return array[deployedRow-250:landedRow+100]

def stateTransitionRows(array):
    rows = []
    for i in range(1,5):
        rows.append(np.where(array[:,0] == i)[0][0])
    return rows

def seaLevel(pressure, altitude):
    return pressure / pow(1.0 - (altitude / 44330.76), 5.25588);

def addAltitudeColumns(array, seaLevel):
    pressureCol = array[:,3]
    altitudeCol = 44330.76 * (1.0 - pow(pressureCol / seaLevel, 1.0 / 5.25588))
    withAlt = np.concatenate((array, np.expand_dims(altitudeCol, 1)), axis=1)
    
    processedAltitude = np.zeros((pressureCol.shape[0], 3))
    for i in range (altitudeCol.shape[0]):
        if (i < 40):
            processedAltitude[i,0] = np.average(altitudeCol[:i+1])
        else:
            processedAltitude[i,0] = np.average(altitudeCol[i-39:i+1])
    processedAltitude[0,1] = 0
    for i in range (1, altitudeCol.shape[0]):
        processedAltitude[i,1] = (processedAltitude[i,0] - processedAltitude[i-1,0]) * 20
    for i in range (altitudeCol.shape[0]):
        if (i < 40):
            processedAltitude[i,2] = np.average(processedAltitude[:i+1, 1])
        else:
            processedAltitude[i,2] = np.average(processedAltitude[i-39:i+1, 1])
    return np.concatenate((withAlt, processedAltitude), axis=1)

def plotData(array, headers, lineColor):
    timeCol = 1
    yColumns = [3, 16, 6, 4, 14, 18, 10, 13]
    numPlots = len(yColumns)
    plotColumns = (numPlots+1)//2
    stateChangeRows = []
    print('State changes at times: ', end=' ')
    for i in range(1,5):
        stateChangeRows.append(np.where(array[:,0] == i)[0][0])
        print(array[stateChangeRows[i-1], timeCol], end='  ')
    print()
    fig, axs = plt.subplots(2, plotColumns, figsize=(16, 6), sharex=True)
    for i in range(numPlots):
        currentAx = axs[i//plotColumns, i%plotColumns]
        currentAx.plot(array[:,timeCol], array[:,yColumns[i]], linewidth=0.5, color=lineColor)
        currentAx.set_title(headers[yColumns[i]])
        for row in stateChangeRows:
            currentAx.axvline(array[row,timeCol], color='black', linewidth=0.5)
            
def plotData2(array, headers, lineColor):
    timeCol = 1
    yColumns = [3, 16, 6, 4, 14, 18, 10, 13]
    numPlots = len(yColumns)
    plotColumns = (numPlots+1)//2
    fig, axs = plt.subplots(2, plotColumns, figsize=(16, 6), sharex=True)
    for i in range(numPlots):
        currentAx = axs[i//plotColumns, i%plotColumns]
        currentAx.plot(array[:,timeCol], array[:,yColumns[i]], linewidth=0.5, color=lineColor)
        currentAx.set_title(headers[yColumns[i]])

In [19]:
dataHeaders = ['State', 'Time (s)', 'Time (ms)', 'Pressure [Pa]', 'Temperature (C)', 'Temperature (C/100)',
               'Accel (G)', 'Accel X', 'Accel Y', 'Accel Z',
               'Batt Sense', 'Cut Sense 1', 'Cut Sense 2', 'Current Sense', 'Photoresistor',
               'Altitude (m)', 'Smoothed Altitude (m)', 'Delta Altitude (m/s)', 'Smoothed Delta Altitude (m/s)']
dataSets = [darthPlagueis, apple]

def blackMagic(array, lineColor, pressure, altitude):
    
    plotData(addAltitudeColumns(filterData(array), seaLevel(pressure, altitude)), dataHeaders, lineColor)
    
def blackMagic2(array, lineColor, pressure, altitude, deployedRow, landedRow):
    plotData2(addAltitudeColumns(filterData2(array, deployedRow, landedRow), seaLevel(pressure, altitude)), dataHeaders, lineColor)

# Matt's flight

TODO   

## Darth Plagueis (PCB 3)

In [20]:
blackMagic(darthPlagueis, 'teal', 96549, 502.00)

State changes at times:  nan  nan  nan  nan  


IndexError: index 18 is out of bounds for axis 1 with size 18

## Apple (PCB 4)

In [ ]:
blackMagic(apple, 'forestgreen', 96595, 500.28)